In [ ]:
import torchvision
import torch
import cv2
import PIL.Image
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
# import ipywidgets.widgets as widgets # JetBot Jupyter 환경에서 사용
# from IPython.display import display # JetBot Jupyter 환경에서 사용
import time
from jetbot import Robot, Camera # JetBot 라이브러리

# --- 모델 정의 및 로드 ---
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

# 모델 가중치 로드 (파일 경로 확인 필요)
try:
    model.load_state_dict(torch.load('best_steering_model_xy.pth'))
except FileNotFoundError:
    print("Error: 'best_steering_model_xy.pth' 파일을 찾을 수 없습니다. 모델 경로를 확인해주세요.")
    exit() # 파일이 없으면 프로그램 종료

# GPU 설정
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half() # 평가 모드 및 반정밀도 사용

# --- 이미지 전처리 설정 ---
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

# --- JetBot 카메라 및 로봇 초기화 ---
camera = Camera.instance(width=224, height=224, fps=10)
robot = Robot()

# --- 제어 변수 초기화 (ipywidgets가 없는 경우 상수 값 사용) ---
# Jupyter Notebook 환경에서 ipywidgets를 사용하지 않는다면 아래 값들을 직접 조정하여 사용하세요.
# 만약 ipywidgets를 사용한다면, 해당 위젯들을 생성하고 이 변수들에 연결해야 합니다.
x_slider_value = 0.0 # 예측 x 좌표 표시용
y_slider_value = 0.0 # 예측 y 좌표 표시용

speed_gain_value = 0.3 # 기본 속도 게인 (속도 조절)
steering_gain_value = 1.0 # 조향 P 게인
steering_dgain_value = 0.1 # 조향 D 게인
steering_bias_value = 0.0 # 조향 바이어스 (직진 보정)

# 현재 각도 및 이전 각도
angle = 0.0
angle_last = 0.0

# --- 메인 루프 ---
print("JetBot 라인 추적 시작. 종료하려면 Ctrl+C를 누르세요.")
try:
    while True:
        frame = camera.value
        frame = cv2.resize(frame, (224, 224))

        # HSV 변환 (모델 학습 데이터에 따라 주석 처리 또는 해제)
        # 일반적으로 BGR/RGB로 학습된 모델을 사용하므로 주석 처리합니다.
        # hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # model(preprocess(hsv_frame)) 대신 model(preprocess(frame)) 사용

        # 모델 예측
        xy = model(preprocess(frame)).detach().float().cpu().numpy().flatten()
        
        x = xy[0]
        y = (0.5 - xy[1]) # y값의 의미를 반전시키는 것으로 보임

        # (디버깅/표시용) 예측된 x, y 값을 슬라이더 변수에 업데이트
        x_slider_value = x
        y_slider_value = y

        # 속도 설정
        current_speed = speed_gain_value # 기본 속도 적용

        # 조향 각도 계산
        angle = np.arctan2(x, y) # x, y로부터 각도 계산

        # PID 제어 (P + D)
        pid = angle * steering_gain_value + (angle - angle_last) * steering_dgain_value
        angle_last = angle # 이전 각도 업데이트

        # 최종 조향 값
        current_steering = pid + steering_bias_value

        # 모터 제어
        # 왼쪽 모터: 기본 속도 + 조향 값
        # 오른쪽 모터: 기본 속도 - 조향 값 (직진을 기준으로 반대 방향으로 조향)
        left_motor_speed = float(np.clip(current_speed + current_steering, 0.0, 1.0))
        right_motor_speed = float(np.clip(current_speed - current_steering, 0.0, 1.0))

        robot.left_motor.value = left_motor_speed
        robot.right_motor.value = right_motor_speed

        # 현재 상태 출력 (디버깅용)
        print(f"Pred XY: ({x:.4f}, {y:.4f}), Angle: {np.degrees(angle):.2f} deg, PID: {pid:.4f}, Steering: {current_steering:.4f}")
        print(f"Left Motor: {left_motor_speed:.4f}, Right Motor: {right_motor_speed:.4f}\n")

        # 짧은 대기 (선택 사항, CPU 사용량 조절)
        # time.sleep(0.01)
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='jpeg')
        img_byte_arr = img_byte_arr.getvalue()

        clear_output(wait=True)
        display(Image.open(io.BytesIO(img_byte_arr)))

except KeyboardInterrupt:
    print("\n키보드 인터럽트 감지. 로봇과 카메라를 정지합니다.")
except Exception as e:
    print(f"\n오류 발생: {e}")
finally:
    # 프로그램 종료 시 로봇 및 카메라 정지
    robot.stop()
    camera.stop()
    print("로봇과 카메라가 성공적으로 정지되었습니다.")